<a href="https://colab.research.google.com/github/JoaoPauloRP/trilha-python-dio/blob/main/notebookbddacab868.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import the necessary packages
!pip3 install imutils
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os
import cv2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Load dataset

In [ ]:
import tensorflow as tf
_URL = 'https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip'
zip_dir = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)

824887076/824887076 [==============================] - 19s 0us/step


In [ ]:
zip_dir

'/root/.keras/datasets/cats_and_dogs.zip'

In [ ]:
%%capture
!unzip /root/.keras/datasets/cats_and_dogs.zip;

In [ ]:
!ls

 CDLA-Permissive-2.0.pdf   PetImages  'readme[1].txt'   sample_data


In [ ]:
# initialize the initial learning rate, number of epochs to train for,
# and batch size
IMG_SIZE = 224
INIT_LR = 1e-4
BS = 256
EPOCHS = 8

In [ ]:
train = ImageDataGenerator(
rescale = 1./255,
horizontal_flip=True,
vertical_flip=True,
shear_range=0.2,
zoom_range=0.2,
rotation_range=40, # Degree range for random rotations
width_shift_range=0.2,
height_shift_range=0.2,
fill_mode='nearest',
validation_split=0.2)

In [ ]:
dest = 'PetImages'

## Removing Corrupt images

In [ ]:
import PIL
from pathlib import Path
from PIL import UnidentifiedImageError

In [ ]:
path = Path('PetImages/Cat').rglob("*.jpg")
for img_p in path:
    try:
        img = PIL.Image.open(img_p)
    except PIL.UnidentifiedImageError:
            print(img_p)

PetImages/Cat/666.jpg


In [ ]:
import os
try: 
    os.remove("PetImages/Cat/666.jpg")
    print("removed")
except:
  print("can't remove") 
  pass

removed


In [ ]:
path = Path('PetImages/Dog').rglob("*.jpg")
for img_p in path:
    try:
        img = PIL.Image.open(img_p)
    except PIL.UnidentifiedImageError:
            print(img_p)

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 270
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5 bytes but only got 0. Skipping tag 271
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 272
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only

PetImages/Dog/11702.jpg


In [ ]:
import os
try: 
    os.remove("PetImages/Dog/11702.jpg")
    print("removed")
except:
  print("can't remove") 
  pass

removed


## Train and Valid Set

In [ ]:
trainset = train.flow_from_directory(dest,
target_size = (IMG_SIZE,IMG_SIZE),
batch_size = BS,
shuffle=False,
seed=42,
color_mode='rgb',
subset = "training",
class_mode='categorical')


Found 20000 images belonging to 2 classes.


In [ ]:
validset = train.flow_from_directory(dest,
target_size = (IMG_SIZE,IMG_SIZE),
batch_size = BS,
shuffle=False,
seed=42,
color_mode='rgb',
subset = "validation",
class_mode='categorical')

Found 4998 images belonging to 2 classes.


In [ ]:
NUM_CLASSES = 2

## Model Architecture - Xception!

In [ ]:
# load the MobileNetV2 network, ensuring the head FC layer sets are
# left off
baseModel = Xception(weights="imagenet", include_top=False,input_tensor=Input(shape=(IMG_SIZE,IMG_SIZE, 3)))


# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(5, 5))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(64, activation="relu")(headModel)
headModel = Dropout(0.8)(headModel)
headModel = Dense(NUM_CLASSES, activation="softmax")(headModel)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False


# compile our model
print("[INFO] compiling model...")
opt = Adam()
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
# train the head of the network
print("[INFO] training head...")

83683744/83683744 [==============================] - 1s 0us/step
[INFO] compiling model...
[INFO] training head...


## Training

In [ ]:
H = model.fit_generator(
	trainset,
  validation_data = validset,
	epochs=EPOCHS)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  after removing the cwd from sys.path.


Epoch 1/8
79/79 [==============================] - 5178s 66s/step - loss: 0.4943 - accuracy: 0.7740 - val_loss: 0.3200 - val_accuracy: 0.8491
Epoch 2/8
79/79 [==============================] - 5176s 66s/step - loss: 0.3274 - accuracy: 0.8669 - val_loss: 0.2502 - val_accuracy: 0.8816
Epoch 3/8
79/79 [==============================] - 5139s 65s/step - loss: 0.2949 - accuracy: 0.8756 - val_loss: 0.2082 - val_accuracy: 0.9164
Epoch 4/8
79/79 [==============================] - 5185s 66s/step - loss: 0.2505 - accuracy: 0.8943 - val_loss: 0.2307 - val_accuracy: 0.8990
Epoch 5/8
79/79 [==============================] - 5131s 65s/step - loss: 0.2390 - accuracy: 0.9000 - val_loss: 0.1916 - val_accuracy: 0.9204
Epoch 6/8
79/79 [==============================] - 5170s 66s/step - loss: 0.2341 - accuracy: 0.9082 - val_loss: 0.1876 - val_accuracy: 0.9230
Epoch 7/8
79/79 [==============================] - 5132s 65s/step - loss: 0.2256 - accuracy: 0.9075 - val_loss: 0.1780 - val_accuracy: 0.9268
Epoch 

In [ ]:
# serialize the model to disk
print("[INFO] saving mask detector model...")
model.save('cat_dog', save_format="h5")

[INFO] saving mask detector model...


NameError: ignored

## Accuracy Loss Plot

In [ ]:
# plot the training loss and accuracy
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.show()
plt.savefig('graph.png')

NameError: ignored